In [1]:
import sys
if "../" not in sys.path: sys.path.insert(0,"../");

In [12]:
## Setting up the needed to do anything
from pathlib import Path
from typing import Union, List, Iterable
import os

from marynlp.text.processors.formatters import lowercase, remove_punctuations, white_space_cleaning
from marynlp import funcutils as f

data_path = Path("../resources/data")
helsinki_na_path = data_path / Path("./hcs-na-v2")

# File to test out the concept
sample_file = helsinki_na_path / Path("./new-mat/bunge/han1-2004.shu")

def lowercase(text):
    return text.lower()

def ignore_rules(text: str):
    return not (text.find("<text") > -1 or text.find("</text>") > -1)

def should_be_longer_that_20(text: str):
    return len(text) > 20

@f.forEach(lowercase)
@f.filterBy(f.rules(should_be_longer_that_20, ignore_rules))
def load_file(file):
    with open(file, "r") as f:
        return f.readlines()
    
def save_to_file(file, content: Iterable[str]):
    with open(file, "w") as wf:
        for line in content:
            wf.write(line)
    
filtered_fn = f.filterBy(f.rules(should_be_longer_that_20, ignore_rules))(load_file)

folder_path = "../resources/operate_on"
!mkdir -p {folder_path}
save_to_file(f'{folder_path}/dummy.txt', filtered_fn(sample_file))

## Regular

In [ ]:
# Working with the fasttext contents

## SED

## Voice